In [3]:
NONE = ' '
WHITE = 'O'
BLACK = 'X'

def opposite_color(color: str) -> str:
    """changes the color"""
    if color == BLACK:
        return WHITE
    else:
        return BLACK

    
from collections import namedtuple
Location = namedtuple('Location', ['col', 'row'])



class Piece:
    def __init__(self, color: str, col: int, row: int):
        """Initializes Piece to have a col, row, and color"""
        self._color = color
        self._col = col
        self._row = row

    def get_row(self) -> int:
        """gets the # of rows"""
        return self._row

    def get_col(self) -> int:
        """gets the # of cols"""
        return self._col

    def get_color(self) -> str:
        """Returns color of this Piece"""
        return self._color

    def flipper(self):
        """flips the color of the pieces"""
        if self._color == WHITE:
            self._color = BLACK
        else:
            self._color = WHITE
        



class Board:
    def __init__(self, cols: int, rows:int, top_left_pos: str):
        """Initializes Board to have a col, row, and color"""
        if (cols%2==0 and cols>=4 and cols<=16 and rows%2==0
            and rows>=4 and rows<=16):
            self._col = cols
            self._row = rows
            self._board = self.board() #saves board changes
            self.starting_positions(top_left_pos)
        else:
            raise ValueError()  #handle error in UI, ask user again

    def get_columns(self):
        """gets column number"""
        return self._col

    def get_rows(self):
        """gets row number"""
        return self._row

    def get_board(self):
        """get board"""
        return self._board
    
    def board(self)->list:
        """Creates new board"""
        
        board = []
        for i in range(self._col):
            row = []
            for j in range(self._row):
                row.append(NONE)
            board.append(row)
        return board

    def starting_positions(self, top_left_pos:str):
        """drops pieces in starting locations"""
        if top_left_pos == BLACK:
            opposite_color = WHITE
        else:
            opposite_color = BLACK
        x = int(self._col/2)
        y = int(self._row/2)
        self.drop_on_board(x-1, y-1, top_left_pos)
        self.drop_on_board(x, y, top_left_pos)
        self.drop_on_board(x, y-1, opposite_color)
        self.drop_on_board(x-1, y, opposite_color)
        

    def drop_on_board(self, col: int, row: int, color: str):
        """Puts a piece on the board, checks if it fits on the board, but
           not if its a valid Othello move"""
        self._board[col][row] = Piece(color, col, row)

    def print_board(self) -> None:
        """
        Prints the current state of the board
        """
        board = self._board
        print(' ', end=' ')
        for i in range(len(board)):
            print('{:2d}'.format(i+1), end = ' ') 
        print()

        for row in range(self._row):
            print('{:2d}'.format(row+1), end= ' ')
            for col in range(self._col):
                box = board[col][row] 
                if(box == NONE):
                    print('{:2s}'.format('.'), end=' ')
                else:
                    print('{:2s}'.format(box.get_color()), end=' ')
            print()

        
    

class Game:
    def __init__(self, board_cols:int, board_rows: int,
                 color:str, top_left_pos:str):
        self._cols = board_cols
        self._rows = board_rows
        self._board = Board(board_cols, board_rows, top_left_pos)
        self._current_player = color

    def get_current_player(self):
        """gets current player"""
        return self._current_player

    def test_drop_valid_piece(self, drop_col: int, drop_row: int, color: str) ->list:
        """
          Test for valid drops
          #1 Test if its valid to drop the piece in that place
        """
        pieces = []
        directions = ['right', 'left', 'top', 'bottom', 'top_right', 'top_left', 'bottom_right', 'bottom_left']
        for direction in directions:
            pieces.extend(self.flip_each_direction(drop_col, drop_row, color, direction))
        return pieces
                      

    def drop_valid_piece(self, drop_col: int, drop_row:int):
        """
          Test for valid drops
          #1 flip the pieces that need to be flipped
          #2 drop the piece
        """
        pieces = self.test_drop_valid_piece(drop_col,drop_row,self._current_player)
        
        if len(pieces) != 0:
            for piece in pieces:
                piece.flipper()
            self._board.drop_on_board(drop_col, drop_row, self._current_player)
            self.flips_no_moves()
        else:
            raise ValueError()

        

    def flip_each_direction(self, starting_col: int, starting_row: int, color: str, direction: str) ->list:
        """Find flippable pieces in each direction, returns list of coordinates in that direction"""
        flippable_pieces = []
        loop_count = 0
        i = starting_col
        j = starting_row
        valid = False
        while True:
            if i == starting_col and j == starting_row:
                piece = self._board.get_board()[i][j]
                if type(piece)== Piece:
                    break
            elif i != starting_col or j != starting_row:
                piece = self._board.get_board()[i][j]
                if type(piece) == str: ##is empty
                    break
                elif loop_count == 1: ## piece next to starting point
                    if piece.get_color() == color:
                        break
                elif piece.get_color() == color:
                    valid = True
                    break

                flippable_pieces.append(piece)

            if direction == 'right':
                if i == self._cols-1:
                    break
                i +=1
            elif direction == 'left':
                if i == 0:
                    break
                i -= 1
            elif direction == 'top':
                if j == 0:
                    break
                j -= 1
            elif direction == 'bottom':
                if j == self._rows-1:
                    break
                j += 1
            elif direction == 'top_right':
                if i == self._cols-1 or j == 0:
                    break
                i += 1
                j -= 1
            elif direction == 'top_left':
                if i == 0 or j == 0:
                    break
                i -= 1
                j -= 1
            elif direction == 'bottom_right':
                if i == self._cols-1 or j == self._rows-1:
                    break
                i += 1
                j += 1
            elif direction == 'bottom_left':
                if i == 0 or j == self._rows - 1:
                    break
                i -= 1
                j += 1

            loop_count += 1
                
        if valid:
            return flippable_pieces
        else:
            return []
        
                                      
        ## 1.) if the FIRST spot has a piece that is the opposite color, then can keep going           
        ## 2.) if blank, it means it cant be valid even. if not, keep going
        ## 3.) if you hit a piece after the first one that is your color, it's a valid move
        

    def num_white_piece(self):
        """counts number of white pieces"""
        count = 0
        for column in self._board.get_board():
            for cell in column:
                if type(cell) == Piece and cell.get_color() == WHITE:
                    count +=1
        return count 

    def num_black_piece(self):
        """counts number of black pieces"""
        count = 0
        for column in self._board.get_board():
            for cell in column:
                if type(cell) == Piece and cell.get_color() == BLACK:
                    count +=1
        return count

    def opposite_turn(self):
        """given the player whose turn it is now, return the opposite player"""
        self._current_player = opposite_color(self._current_player)

    
    def list_of_valid_moves(self, color: str):
        """Returns a list of valid moves that a player has"""
        valid = []
        for col in range(self._cols):
            for row in range(self._rows):
                if type(self._board.get_board()[col][row]) == str:
                    flippable_pieces = self.test_drop_valid_piece(col, row, color)
                    if len(flippable_pieces) != 0:
                        valid.append(Location(col=col+1, row=row+1))
        return valid

    def flips_no_moves(self):
        """If the player has no more moves, flip to the next player.
           If the new player has no more moves end game(throw error)."""
        current_player_moves = self.list_of_valid_moves(self._current_player)
        next_color = opposite_color(self._current_player)
        opposite_player_moves = self.list_of_valid_moves(next_color)

        if len(opposite_player_moves) != 0:
            self.opposite_turn()
        elif len(opposite_player_moves) == 0 and len(current_player_moves) == 0:
            ## both players can't move; game over
            raise GameOverError()

class GameOverError(Exception):
    pass


In [4]:
import tkinter
#import othello_logic


        
class OthelloOptions:
    def __init__(self):
        """Initialize all of the board options"""
        #size of window
        self._size_window = tkinter.Tk()
        self._size_window.title("Inputs Window")
 
        options = (4,6,8,10,12,14,16)
        self._usercol = tkinter.StringVar(self._size_window)
        self._usercol.set(options[0])
        self._col = tkinter.OptionMenu(self._size_window,self._usercol, *options)
        self._col.grid(row = 0, column = 1,
                       sticky = tkinter.N + tkinter.S + tkinter.W + tkinter.E)
        
        self._userrow = tkinter.StringVar(self._size_window)
        self._userrow.set(options[0])
        self._row = tkinter.OptionMenu(self._size_window,self._userrow, *options)
        self._row.grid(row =1, column =1,
                       sticky = tkinter.N + tkinter.S + tkinter.W + tkinter.E)
        

        self._col_label = tkinter.Label(self._size_window, text = 'Number of Columns: ')
        self._col_label.grid(row =0, column =0,
                       sticky = tkinter.W)

        self._row_label = tkinter.Label(self._size_window, text = 'Number of Rows: ')
        self._row_label.grid(row =1, column =0,
                       sticky = tkinter.W)

            
        
        self._size_window.rowconfigure(2, weight = 5)
        self._size_window.rowconfigure(5, weight = 5)
        self._size_window.rowconfigure(8, weight = 5)
        self._size_window.rowconfigure(11, weight = 5)
        self._size_window.columnconfigure(0, weight = 5)
        self._size_window.columnconfigure(1, weight = 5)
       
        
        #which player moves first button
        self._move_first = tkinter.StringVar(self._size_window)

        self._move_first.set(othello_logic.WHITE)
        self._first_button = tkinter.Radiobutton(self._size_window, variable = self._move_first, text = 'White',value=othello_logic.WHITE)
        self._second_button = tkinter.Radiobutton(self._size_window, variable = self._move_first, text = 'Black',value=othello_logic.BLACK)
        
        self._first_button.grid(row = 3, column = 1,sticky = tkinter.N + tkinter.S + tkinter.W + tkinter.E)
        self._second_button.grid(row = 4, column = 1,sticky = tkinter.N + tkinter.S + tkinter.W + tkinter.E)
        
        self._move_label = tkinter.Label(self._size_window, text = 'Which player moves first: ')
        self._move_label.grid(row =3, column =0, rowspan = 2,
                       sticky = tkinter.W)
        #top left player button
        self._top_left = tkinter.StringVar(self._size_window)

        self._top_left.set(othello_logic.WHITE)
        self._third_button = tkinter.Radiobutton(self._size_window, variable = self._top_left, text = 'White',value=othello_logic.WHITE)
        self._fourth_button = tkinter.Radiobutton(self._size_window, variable = self._top_left, text = 'Black',value=othello_logic.BLACK)
        
        self._third_button.grid(row = 6, column = 1,sticky = tkinter.N + tkinter.S + tkinter.W + tkinter.E)
        self._fourth_button.grid(row = 7, column = 1,sticky = tkinter.N + tkinter.S + tkinter.W + tkinter.E)
        
        self._move_label = tkinter.Label(self._size_window, text = 'Which player in top\n left position: ')
        self._move_label.grid(row =6, column =0, rowspan = 2,
                       sticky = tkinter.W)
        
        #how to win button
        self._winner = tkinter.StringVar(self._size_window)

        self._winner.set('Most')
        self._fifth_button = tkinter.Radiobutton(self._size_window, variable = self._winner, text = 'Most',value= 'Most')
        self._sixth_button = tkinter.Radiobutton(self._size_window, variable = self._winner, text = 'Least',value= 'Least')
        
        self._fifth_button.grid(row = 9, column = 1,sticky = tkinter.N + tkinter.S + tkinter.W + tkinter.E)
        self._sixth_button.grid(row = 10, column = 1,sticky = tkinter.N + tkinter.S + tkinter.W + tkinter.E)
        
        self._move_label = tkinter.Label(self._size_window, text = 'Win by most or least: ')
        self._move_label.grid(row =9, column =0, rowspan = 2,
                       sticky = tkinter.W)
        
        #start button, destroy window
        self._start = tkinter.Button(self._size_window, text = 'PLAY GAME!')
        self._start.grid( row = 12, column = 0, columnspan=2, rowspan=2, padx=50,pady=15,
                          sticky = tkinter.N + tkinter.S + tkinter.W + tkinter.E)

        self._start.bind('<Button-1>',self._on_play_game)


        self._size_window.mainloop()

    def _on_play_game(self,event:tkinter.Event)->None:
        """starts othelloapplication board"""
        self._size_window.destroy()

        OthelloApplication(int(self._usercol.get()),int(self._userrow.get()),self._move_first.get(),
                           self._top_left.get(),self._winner.get())
        

class OthelloApplication:
    def __init__(self, columns: int, rows: int, move_first:str,top_left:str,win_type:str)->None:
        
        #main window
        self._root_window = tkinter.Tk()
        self._root_window.title("Othello Game")

        self._columns = columns
        self._rows = rows
        self._move_first = move_first
        self._top_left = top_left
        self._win_type = win_type
        self._game_over = False
        
        self._othello_game = othello_logic.Game(columns,rows,move_first, top_left)
        self._current_board = self._othello_game._board

        self._canvas = tkinter.Canvas(
            master = self._root_window,
            width = 500, height = 500,
            background = 'darkgreen')
        self._canvas.grid(row = 1, column = 0,
                          sticky = tkinter.N + tkinter.S + tkinter.W + tkinter.E)
        
        self._score = tkinter.Canvas(
            master = self._root_window, width = 500, height = 75, background = 'darkgreen',
            borderwidth = 10 , relief ='raised')
        self._score.grid(
            row = 0, column = 0, pady = 10,
            sticky = tkinter.N + tkinter.W + tkinter.E)


        self._canvas.bind('<Configure>', self._on_canvas_resized)
        self._canvas.bind('<Button-1>', self._on_canvas_clicked)

        self._root_window.rowconfigure(1, weight = 100)
        self._root_window.rowconfigure(0, weight = 5)
        self._root_window.columnconfigure(0, weight = 5)

    def _on_canvas_clicked(self, event:tkinter.Event)-> None:
        """find the coordinates of where the user clicked"""
        width = self._canvas.winfo_width()
        height = self._canvas.winfo_height()
        column_width = int(width/self._columns)
        row_width = int(height/self._rows)
    
        cell_column = int(event.x/column_width)
        cell_row = int(event.y/row_width)
        
        
        self._keep_score()
        #self._current_board.print_board()
        try:
            self._othello_game.drop_valid_piece(cell_column, cell_row)
        except ValueError:
            print('Invalid Move')
        except othello_logic.GameOverError:
            print('Game Over')
            self._game_over = True
            self._canvas.unbind('<Button-1>')
  
        
        #self._current_board.print_board()

        self._draw_circles()
        self._keep_score()
        
    def _on_canvas_resized(self, event: tkinter.Event) -> None:
        """Resize objects on the board"""
        print('resizing')
        self._canvas.delete(tkinter.ALL)
        
        self._draw_lines()
        self._draw_circles()
        self._keep_score()

    def _draw_lines(self)->None:
        """draw the board lines"""
        canvas_width = self._canvas.winfo_width()
        canvas_height = self._canvas.winfo_height()
        column_width = int(canvas_width/self._columns)
        row_width = int(canvas_height/self._rows)

        
        #column lines
        for i in range(0,self._columns):
            self._canvas.create_line(i*(column_width), 0, i*(column_width), canvas_height)
        #row lines
        for i in range(0, self._rows):
            self._canvas.create_line( 0, i*row_width, canvas_width, i*row_width)
    
    def _draw_circles(self)->None:
        """draw the pieces based on game logic"""
        width = self._canvas.winfo_width()
        height = self._canvas.winfo_height()
        column_width = int(width/self._columns)
        row_width = int(height/self._rows)
    
        
        board = self._current_board.get_board()

        for column in board:
            for cell in column:
                if type(cell) == othello_logic.Piece:
                    cell_column = cell.get_col()
                    cell_row = cell.get_row()
                    if cell.get_color() == othello_logic.BLACK:
                        self._canvas.create_oval((cell_column*column_width,
                                                 cell_row*row_width, (cell_column+1)*column_width,
                                                 (cell_row + 1)*row_width), fill = 'black')
                    elif cell.get_color() == othello_logic.WHITE:
                        self._canvas.create_oval((cell_column*column_width,
                                                 cell_row*row_width, (cell_column+1)*column_width,
                                                 (cell_row + 1)*row_width), fill = 'white')
        
    def _keep_score(self)-> None:
        """keep the score of the number of pieces, determine turn, display winner"""
        self._score.delete(tkinter.ALL)
        width = self._score.winfo_width()
        height = self._score.winfo_height()
        win_type = self._win_type
        
        white = self._othello_game.num_white_piece()
        black = self._othello_game.num_black_piece()
        current_player =  self._othello_game.get_current_player()

        if not self._game_over:
            self._score.create_text( width/2, height/2, text = get_turn(current_player),
                                     fill = 'white', font =("Purisa",27) )
        else:
            if  white == black:               
                self._score.create_text( width/2, height/2, text = 'TIE GAME',
                                     fill = 'white', font =("Purisa",27) )
            elif win_type == 'Most':
                if white > black:
                    self._score.create_text( width/2, height/2, text = 'Winner is WHITE!!!',
                                     fill = 'white', font =("Purisa",27) )
                elif black > white:
                    self._score.create_text( width/2, height/2, text = 'Winner is BLACK!!!',
                                     fill = 'white', font =("Purisa",27) )
            elif win_type == 'Least':
                if white < black:
                    self._score.create_text( width/2, height/2, text = 'Winner is WHITE!!!',
                                     fill = 'white', font =("Purisa",27) )
                elif black < white:
                    self._score.create_text( width/2, height/2, text = 'Winner is BLACK!!!',
                                     fill = 'white', font =("Purisa",27) )
        self._score.create_oval( 0+15,0+15,0+85, height-15, fill = 'white')
        self._score.create_oval( width-85, 0+15, width-15, height-15, fill = 'black')
        
        self._score.create_text( 50, 50, text = white,
                                 fill = 'black', font =("Purisa",27) )
        self._score.create_text( width-50, 50, text = black,
                                 fill = 'white', font =("Purisa",27) )
        #print(white, black, current_player)
        
        
    def start(self) -> None:
        """ Starts the Othello application. Will close when window is closed"""
        self._root_window.mainloop()

    


def get_turn(current_player: str)->str:
    """Prints turn on board"""
    if current_player == 'O':
        return ("white's turn")
    elif current_player == 'X':
        return ("black's turn")
    

if __name__ == '__main__':
    OthelloOptions()

NameError: name 'othello_logic' is not defined